In [0]:
import pandas as pd

# Load dari Delta table
df_spark = spark.table("cross_sell_insurance.01_feature_staging.stage1_early_feature_table")

df_spark.printSchema()
df_spark.show(5)


In [0]:
df = df_spark.toPandas()

df.head()

In [0]:
df.info()

In [0]:
df.describe(include='all')

In [0]:
import pandas as pd

missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]

missing


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
missing.plot(kind='bar')
plt.title("Missing Values per Column")
plt.show()

In [0]:
# Imbalance Check
df['is_target_customer'].value_counts()
df['is_target_customer'].value_counts().plot(kind='bar')
plt.title("Target Distribution")
plt.xticks(rotation=0)
plt.show()


In [0]:
# Hitung jumlah dan persentase tiap kelas
counts = df["is_target_customer"].value_counts()
percentages = df["is_target_customer"].value_counts(normalize=True) * 100

imbalance_table = pd.DataFrame({
    "count": counts,
    "percentage": percentages.round(2)
})

print(imbalance_table)


In [0]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

df[numeric_cols].hist(figsize=(16,12), bins=30)
plt.tight_layout()
plt.show()


In [0]:
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    plt.figure(figsize=(6,3))
    df[col].value_counts().head(10).plot(kind='bar')
    plt.title(f"Top Values in {col}")
    plt.show()


In [0]:
import seaborn as sns

plt.figure(figsize=(16,12))
corr = df[numeric_cols].corr()

sns.heatmap(corr, cmap='coolwarm', annot=False)
plt.title("Correlation Heatmap")
plt.show()


In [0]:
for col in numeric_cols[:8]:  # batasi dulu biar tidak kebanyakan
    plt.figure(figsize=(5,3))
    sns.boxplot(x=df['is_target_customer'], y=df[col])
    plt.title(f"{col} vs Target")
    plt.show()


In [0]:
for col in categorical_cols[:8]:
    target_rate = df.groupby(col)['is_target_customer'].mean().sort_values(ascending=False)
    print("\n", col)
    print(target_rate.head(10))


In [0]:
# Summary
# Birth Date is not necessary since there are columns for age sparsed.--> drop Birth Date
# Insured Age is not necessary since it is empty.--> drop Insured Age
# Correlation analysis: Policy counts is highly correlated with agency_counts --> drop Policy counts
# Need to one Hot Encode for the Object columns --> SKLearn Pipeline
# Need to Log Transform and autoscale for the numeric columns --> SKLearn Pipeline
# Need to Impute for the missing values --> SKLearn Pipeline
#

In [0]:
%sql
CREATE OR REPLACE TABLE cross_sell_insurance.01_feature_staging.stage2_clean_feature_table AS
SELECT 
    *
EXCEPT (birth_date, insured_age, policy_counts)
FROM cross_sell_insurance.01_feature_staging.stage1_early_feature_table;